### Install keras and tensorflow:

In [1]:
!conda install -c conda-forge keras --yes
!conda install -c conda-forge tensorflow --yes

### Import libraries:

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import os

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

/Users/martinbrandt/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/martinbrandt/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/martinbrandt/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/martinbrandt/opt/anaconda3/lib/python3.7/s

### Data import:

In [2]:
folder_path = 'food/'

img_width, img_height = 224, 224

# load all images into a list
file_list = sorted(os.listdir(folder_path))
images = []
for img in file_list:
    if img == '.DS_Store':  # ignore stupid fookin mac file that wont go away
        continue
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    images.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)

### Load pre-trained model:

In [3]:
model = ResNet50(weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.


### Predict:

In [4]:
predictions = model.predict(images, batch_size=64, verbose=1)

10000/10000 [==============================] - 3732s 373ms/sample


### Save:

In [5]:
# Save top 3 label probabilities as the 3 features describing image:
predictions_decoded = np.array(decode_predictions(predictions, top=3))
predictions_decoded = predictions_decoded[:, :, 1]

df = pd.DataFrame(predictions_decoded, columns=['first', 'second', 'third'])
print(df)

df.to_csv('food_features', index=False)

           first         second         third
0        thimble      trilobite        packet
1         potpie         bakery         dough
2      ice_cream  mashed_potato         dough
3      ice_cream          dough     meat_loaf
4        burrito        pretzel     carbonara
...          ...            ...           ...
9995   ice_cream     neck_brace      lipstick
9996  strawberry      ice_cream      baseball
9997   meat_loaf          pizza  wooden_spoon
9998   soup_bowl     frying_pan         loupe
9999       dough           wool        mortar

[10000 rows x 3 columns]


In [6]:
'''img_path = 'food/00000.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

pred = model.predict(x)
pred = np.array(decode_predictions(pred, top=3))
print(pred)'''

[[['n04423845' 'thimble' '0.7259098']
  ['n01768244' 'trilobite' '0.102758445']
  ['n03871628' 'packet' '0.019814285']]]
